# Install Dependencies

For now only CPU runtime, some problems with GPU

In [ ]:
!git clone https://github.com/photosartd/bce.git
%cd bce
!git fetch origin dev
!git checkout dev
#%env PYTHONPATH=/env/python:/content/bce
!pip install -q -e .

fatal: destination path 'bce' already exists and is not an empty directory.
/content/bce
From https://github.com/photosartd/bce
 * branch            dev        -> FETCH_HEAD
Already on 'dev'
Your branch is up to date with 'origin/dev'.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.0 which is incompatible.


In [ ]:
%cd bce

import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -q -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -q -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -q -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -q -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric -q

!pip install wandb==0.13.3 -q

!pip install torch-geometric-temporal -q

[Errno 2] No such file or directory: 'bce'
/content/bce
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-bce 0.1.0 requires numpy>=1.23.0, but you have numpy 1.22.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-bce 0.1.0 requires numpy>=1.23.0, but you have numpy 1.22.4 which is incompatible.
     |████████████████████████████████| 48 kB 2.7 MB/s 


**Restart runtime**

# Process

In [ ]:
import logging
from typing import Iterable
from functools import cached_property
from tempfile import TemporaryDirectory
from copy import deepcopy

import torch.cuda
import torch.nn as nn
from torch.utils.data import DataLoader
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader, WikiMathsDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

from torch_bce.models import GraphSAGE, MLPRegressor
from torch_bce.losses import GraphSageLoss
from torch_bce.utils.datasets import TensorSupervisedDataset
from torch_bce.trainers.gs_alignment_trainer import GSAlignmentTrainer

In [ ]:
KWARGS = {
    "lambda_": 2,
    "alignment": "multi_step",
    "backward_transformation": "linear",
    "level": logging.INFO,
    "num_epochs": 100,
    "setup_wandb": True
}
DEVICE = "cpu" #some problems with GPU for now

Load temporal graph dataset:

In [ ]:
loader = WikiMathsDatasetLoader()  # ChickenpoxDatasetLoader()
dataset = loader.get_dataset()
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

Set up **Intended Model**

In [ ]:
num_node_features = train_dataset[0].num_node_features
"""Intended model"""
hid_channels = 128
out_channels = 32
intended_model = GraphSAGE(
    loss=GraphSageLoss(),
    in_channels=num_node_features,
    hid_channels=hid_channels,
    out_channels=out_channels,
    n_layers=2,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

Set up **Unintended Model**

In [ ]:
"""Unintended model"""
unintended_model = MLPRegressor(loss=nn.MSELoss(), input_size=out_channels, output_size=1, div=3)
unintended_opt = torch.optim.Adam(unintended_model.parameters(), lr=0.001)

Set up **Trainer**

In [ ]:
local_kwargs = deepcopy(KWARGS)
local_kwargs["setup_wandb"] = True
trainer = GSAlignmentTrainer(
    model=intended_model,
    **local_kwargs
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dimaks27. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Train Intended Model for several epochs:

In [ ]:
N_EPOCHS = 11

In [ ]:
"""Training Intended"""
optimizer = torch.optim.Adam(intended_model.parameters(), lr=3e-4, weight_decay=4e-5)
losses = []
for epoch in range(1, N_EPOCHS):
    loss = 0
    time: int = 1
    for time, data in enumerate(train_dataset, 1):
        model_loss, statistics = trainer.train(
            train_data=data,
            val_data=test_dataset[0],
            walk_length=3,
            sizes=(5, 2),
            batch_size=256,
            shuffle=False,
            log_stats=False
        )
        loss = loss + model_loss
    loss = loss / time
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    trainer.log_metrics(epoch, statistics)
"""Get predictions for latest snapshot"""
predictions = []
ys = []
for data in train_dataset:
    curr_predictions, stats = trainer.inference(
        data,
        walk_length=3,
        sizes=(5, 2),
        batch_size=256,
        shuffle=False,
        log_stats=False
    )
    predictions.append(curr_predictions)
    ys.append(data.y)
predictions = torch.vstack(predictions)
ys = torch.hstack(ys)

predictions_test = []
ys_test = []
for data in test_dataset:
    curr_predictions_test, stats_test = trainer.inference(
        data,
        walk_length=3,
        sizes=(5, 2),
        batch_size=256,
        shuffle=False,
        log_stats=False
    )
    predictions_test.append(curr_predictions_test)
    ys_test.append(data.y)
predictions_test = torch.vstack(predictions_test)
ys_test = torch.hstack(ys_test)

Train Unintended model on embeddings:

In [ ]:
N_EPOCHS = 20

In [ ]:
"""Training unintended"""
unintended_dataset = TensorSupervisedDataset(
    x=predictions,
    y=ys
)
unintended_dataloader = DataLoader(unintended_dataset, batch_size=256, shuffle=False)

unintended_dataset_val = TensorSupervisedDataset(
    x=predictions_test,
    y=ys_test
)
unintended_dataloader_val = DataLoader(unintended_dataset_val, batch_size=256, shuffle=False)
unintended_losses = []
for epoch in range(1, N_EPOCHS):
    loss, metrics = unintended_model.train_loop(unintended_dataloader, unintended_opt)
    preds_val, metrics_val = unintended_model.predict(unintended_dataloader_val)
    trainer.log_metrics(
        epoch=epoch,
        metrics={
            "Unintended: train loss": metrics["mean_train_loss"],
            "Unintended: val loss": metrics_val["mean_loss"],
            "Unintended: MSE train": metrics["MeanSquaredError"],
            "Unintended: MSE val": metrics_val["MeanSquaredError"],
            "Unintended: MAE train": metrics["MeanAbsoluteError"],
            "Unintended: MAE val": metrics_val["MeanAbsoluteError"]
        }
    )
    unintended_losses.append(metrics["mean_train_loss"])

Time to create, replace in trainer and train 2-nd Intended Model:

In [ ]:
"""Intended model 2"""
intended_model_new = GraphSAGE(
    loss=GraphSageLoss(),
    in_channels=num_node_features,
    hid_channels=hid_channels // 2,
    out_channels=out_channels,
    n_layers=3,
    device="cuda" if torch.cuda.is_available() else "cpu"
)
optimizer = torch.optim.Adam(intended_model_new.parameters(), lr=3e-4, weight_decay=4e-5)
"""Replace model"""
trainer.replace_model(
    intended_model_new,
    train_data=test_dataset[0],
    walk_length=3,
    sizes=(5, 2),
    batch_size=256,
    shuffle=False,
    log_stats=False
)
"""Train intended again"""
losses = []
scheduler = torch.optim.lr_scheduler.StepLR(trainer.alignment_optimizer, step_size=50, gamma=0.1)
for epoch in range(1, N_EPOCHS + 70):
    loss2 = 0
    time: int = 1
    for time, data in enumerate(train_dataset, 1):
        model_loss, statistics = trainer.train(
            train_data=data.to(DEVICE),
            val_data=test_dataset[0].to(DEVICE),
            walk_length=3,
            sizes=(5, 2, 1),
            batch_size=256,
            shuffle=False,
            log_stats=False
        )
        loss2 = loss2 + model_loss
    loss2 = loss2 / time
    losses.append(loss2.item())
    loss2.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    """Get preds on step"""
    predictions_test = []
    ys_test = []
    for data in test_dataset:
        curr_predictions_test, stats_test = trainer.inference(
            data,
            walk_length=3,
            sizes=(5, 2, 1),
            batch_size=256,
            shuffle=False,
            log_stats=False
        )
        predictions_test.append(curr_predictions_test)
        ys_test.append(data.y)
    predictions_test = torch.vstack(predictions_test)
    ys_test = torch.hstack(ys_test)

    unintended_dataset_val = TensorSupervisedDataset(
        x=predictions_test,
        y=ys_test
    )
    unintended_dataloader_val = DataLoader(unintended_dataset_val, batch_size=256, shuffle=False)
    preds_val, metrics_val = unintended_model.predict(unintended_dataloader_val)
    trainer.log_metrics(
        epoch=epoch,
        metrics={
            "Unintended: train loss": metrics["mean_train_loss"],
            "Unintended: val loss": metrics_val["mean_loss"],
            "Unintended: MSE train": metrics["MeanSquaredError"],
            "Unintended: MSE val": metrics_val["MeanSquaredError"],
            "Unintended: MAE train": metrics["MeanAbsoluteError"],
            "Unintended: MAE val": metrics_val["MeanAbsoluteError"],
            **statistics
        }
    )